In [14]:
import torch
import torch.nn as nn
import torch.optim as optim
from transformers import AutoModel, AutoTokenizer
import numpy as np
import pandas as pd
import re
import string
import emoji
from nltk import flatten
from datasets import load_dataset
import torch
import torch.nn as nn
import torch.optim as optim
from transformers import AutoModel, AutoTokenizer
import numpy as np
import pandas as pd
import re
import string
import emoji
from nltk import flatten
from datasets import load_dataset
from torch.utils.data import DataLoader, Dataset

In [15]:
import os
from vncorenlp import VnCoreNLP

In [16]:
def remove_HTML(text):
    return re.sub(r'<[^>]*>', '', text) # Thay thế các ký tự ở param 1 bằng ký tự ở param 2 trong chuỗi ký tự text


# Chuẩn hoá unicode, chuyển các ký tự từ mã window 1252 về dạng utf8
def convert_unicode(text):
    char1252 = 'à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ'
    charutf8 = 'à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ'
    char1252 = char1252.split('|')
    charutf8 = charutf8.split('|')

    dic = {}
    for i in range(len(char1252)): dic[char1252[i]] = charutf8[i]
    return re.sub(
        r'à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ',
        lambda x: dic[x.group()], text
    )


# Chuẩn hoá cách typing. Trả ra từ điển vowels_to_ids để tìm ra vị trí của bất kỳ nguyên âm có dấu nào trong bảng vowels_table và thực hiện các thao tác xử lý văn bản tiếp theo dựa trên đó
vowels_to_ids = {}
vowels_table = [
    ['a', 'à', 'á', 'ả', 'ã', 'ạ', 'a' ],
    ['ă', 'ằ', 'ắ', 'ẳ', 'ẵ', 'ặ', 'aw'],
    ['â', 'ầ', 'ấ', 'ẩ', 'ẫ', 'ậ', 'aa'],
    ['e', 'è', 'é', 'ẻ', 'ẽ', 'ẹ', 'e' ],
    ['ê', 'ề', 'ế', 'ể', 'ễ', 'ệ', 'ee'],
    ['i', 'ì', 'í', 'ỉ', 'ĩ', 'ị', 'i' ],
    ['o', 'ò', 'ó', 'ỏ', 'õ', 'ọ', 'o' ],
    ['ô', 'ồ', 'ố', 'ổ', 'ỗ', 'ộ', 'oo'],
    ['ơ', 'ờ', 'ớ', 'ở', 'ỡ', 'ợ', 'ow'],
    ['u', 'ù', 'ú', 'ủ', 'ũ', 'ụ', 'u' ],
    ['ư', 'ừ', 'ứ', 'ử', 'ữ', 'ự', 'uw'],
    ['y', 'ỳ', 'ý', 'ỷ', 'ỹ', 'ỵ', 'y' ]
]

for i in range(len(vowels_table)):
    for j in range(len(vowels_table[i]) - 1):
        vowels_to_ids[vowels_table[i][j]] = (i, j)

#Kiểm tra có phải chữ tiếng Việt không dựa trên từ điển vowels_to_ids
def is_valid_vietnamese_word(word):
    chars = list(word)
    vowel_indexes = -1
    for index, char in enumerate(chars):
        x, y = vowels_to_ids.get(char, (-1, -1))
        if x != -1:
            if vowel_indexes == -1: vowel_indexes = index
            else:
                if index - vowel_indexes != 1: return False
                vowel_indexes = index
    return True

# Chuẩn hoá chữ viết
def standardize_word_typing(word):
    if not is_valid_vietnamese_word(word): return word
    chars = list(word)
    dau_cau = 0
    vowel_indexes = []
    qu_or_gi = False

    for index, char in enumerate(chars):
        x, y = vowels_to_ids.get(char, (-1, -1))
        if x == -1: continue
        elif x == 9:  # check phụ âm 'qu'
            if index != 0 and chars[index - 1] == 'q':
                chars[index] = 'u'
                qu_or_gi = True
        elif x == 5:  # check phụ âm 'gi'
            if index != 0 and chars[index - 1] == 'g':
                chars[index] = 'i'
                qu_or_gi = True

        if y != 0:
            dau_cau = y
            chars[index] = vowels_table[x][0]

        if not qu_or_gi or index != 1:
            vowel_indexes.append(index)

    if len(vowel_indexes) < 2:
        if qu_or_gi:
            if len(chars) == 2:
                x, y = vowels_to_ids.get(chars[1])
                chars[1] = vowels_table[x][dau_cau]
            else:
                x, y = vowels_to_ids.get(chars[2], (-1, -1))
                if x != -1: chars[2] = vowels_table[x][dau_cau]
                else: chars[1] = vowels_table[5][dau_cau] if chars[1] == 'i' else vowels_table[9][dau_cau]
            return ''.join(chars)
        return word

    for index in vowel_indexes:
        x, y = vowels_to_ids[chars[index]]
        if x == 4 or x == 8:  # ê, ơ
            chars[index] = vowels_table[x][dau_cau]
            return ''.join(chars)

    if len(vowel_indexes) == 2:
        if vowel_indexes[-1] == len(chars) - 1:
            x, y = vowels_to_ids[chars[vowel_indexes[0]]]
            chars[vowel_indexes[0]] = vowels_table[x][dau_cau]
        else:
            x, y = vowels_to_ids[chars[vowel_indexes[1]]]
            chars[vowel_indexes[1]] = vowels_table[x][dau_cau]
    else:
        x, y = vowels_to_ids[chars[vowel_indexes[1]]]
        chars[vowel_indexes[1]] = vowels_table[x][dau_cau]
    return ''.join(chars)

# Chuẩn hoá câu viết
def standardize_sentence_typing(text):
    words = text.lower().split()
    for index, word in enumerate(words):
        cw = re.sub(r'(^\p{P}*)([p{L}.]*\p{L}+)(\p{P}*$)', r'\1/\2/\3', word).split('/')
        if len(cw) == 3: cw[1] = standardize_word_typing(cw[1])
        words[index] = ''.join(cw)
    return ' '.join(words)


replace_list = {
    'ô kêi': 'ok', 'okie': 'ok', 'o kê': 'ok', 'okey': 'ok', 'ôkê': 'ok', 'oki': 'ok', 'oke': 'ok', 'okay': 'ok', 'okê': 'ok',
    'tks': 'cảm ơn', 'thks': 'cảm ơn', 'thanks': 'cảm ơn', 'ths': 'cảm ơn', 'thank': 'cảm ơn',
    'kg': 'không', 'not': 'không', 'k': 'không', 'kh': 'không', 'kô': 'không', 'hok': 'không', 'ko': 'không', 'khong': 'không', 'kp': 'không phải',
    'he he': 'tích cực', 'hehe': 'tích cực', 'hihi': 'tích cực', 'haha': 'tích cực', 'hjhj': 'tích cực', 'thick': 'tích cực',
    'lol': 'tiêu cực', 'cc': 'tiêu cực', 'huhu': 'tiêu cực', 'cute': 'dễ thương',

    'sz': 'cỡ', 'size': 'cỡ',
    'wa': 'quá', 'wá': 'quá', 'qá': 'quá',
    'đx': 'được', 'dk': 'được', 'dc': 'được', 'đk': 'được', 'đc': 'được',
    'vs': 'với', 'j': 'gì', '“': ' ', 'time': 'thời gian', 'm': 'mình', 'mik': 'mình', 'r': 'rồi', 'bjo': 'bao giờ', 'very': 'rất',

    'authentic': 'chuẩn chính hãng', 'aut': 'chuẩn chính hãng', 'auth': 'chuẩn chính hãng', 'date': 'hạn sử dụng', 'hsd': 'hạn sử dụng',
    'store': 'cửa hàng', 'sop': 'cửa hàng', 'shopE': 'cửa hàng', 'shop': 'cửa hàng',
    'sp': 'sản phẩm', 'product': 'sản phẩm', 'hàg': 'hàng',
    'ship': 'giao hàng', 'delivery': 'giao hàng', 'síp': 'giao hàng', 'order': 'đặt hàng',

    'gud': 'tốt', 'wel done': 'tốt', 'good': 'tốt', 'gút': 'tốt', 'tot': 'tốt', 'nice': 'tốt', 'perfect': 'rất tốt',
    'quality': 'chất lượng', 'chất lg': 'chất lượng', 'chat': 'chất', 'excelent': 'hoàn hảo', 'bt': 'bình thường',
    'sad': 'tệ', 'por': 'tệ', 'poor': 'tệ', 'bad': 'tệ',
    'beautiful': 'đẹp tuyệt vời', 'dep': 'đẹp',
    'xau': 'xấu', 'sấu': 'xấu',

    'thik': 'thích', 'iu': 'yêu', 'fake': 'giả mạo',
    'quickly': 'nhanh', 'quick': 'nhanh', 'fast': 'nhanh',
    'fresh': 'tươi', 'delicious': 'ngon',

    'dt': 'điện thoại', 'fb': 'facebook', 'face': 'facebook', 'ks': 'khách sạn', 'nv': 'nhân viên',
    'nt': 'nhắn tin', 'ib': 'nhắn tin', 'tl': 'trả lời', 'trl': 'trả lời', 'rep': 'trả lời',
    'fback': 'feedback', 'fedback': 'feedback',
    'sd': 'sử dụng', 'sài': 'xài',

    '^_^': 'tích cực', ':)': 'tích cực', ':(': 'tiêu cực',
    '❤️': 'tích cực', '👍': 'tích cực', '🎉': 'tích cực', '😀': 'tích cực', '😍': 'tích cực', '😂': 'tích cực', '🤗': 'tích cực', '😙': 'tích cực', '🙂': 'tích cực',
    '😔': 'tiêu cực', '😓': 'tiêu cực',
    '⭐': 'star', '*': 'star', '🌟': 'star',
}

# Chuẩn hoá các từ viết tắt, ký tự thành tiếng Việt dựa trên danh sách replace_list
def normalize_acronyms(text):
    words = []
    for word in text.strip().split():
        # word = word.strip(string.punctuation)
        if word.lower() not in replace_list.keys(): words.append(word)
        else: words.append(replace_list[word.lower()])
    return emoji.demojize(' '.join(words)) # Remove Emojis


# Word segmentation: Trả ra một chuỗi văn bản với các từ đã được phân tách và nối lại với nhau bằng khoảng trắng, sử dụng nguồn file chuẩn hoá VnCoreNLP-1.1.1.jar
#annotator = VnCoreNLP('VnCoreNLP/VnCoreNLP-1.1.1.jar')
#def word_segmentation(text):
#    words = annotator.tokenize(text)
#    return ' '.join(word for word in flatten(words))


# Loại bỏ các ký tự không cần thiết trong văn bản
def remove_unnecessary_characters(text):
    text = re.sub(r'[^\s\wáàảãạăắằẳẵặâấầẩẫậéèẻẽẹêếềểễệóòỏõọôốồổỗộơớờởỡợíìỉĩịúùủũụưứừửữựýỳỷỹỵđÁÀẢÃẠĂẮẰẲẴẶÂẤẦẨẪẬÉÈẺẼẸÊẾỀỂỄỆÓÒỎÕỌÔỐỒỔỖỘƠỚỜỞỠỢÍÌỈĨỊÚÙỦŨỤƯỨỪỬỮỰÝỲỶỸỴĐ_]', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip() # Remove extra whitespace
    return text
# Tạo function tổng hợp xử lý 1 văn bản
def text_preprocess(text):
    text = remove_HTML(text)
    text = convert_unicode(text)
    # text = standardize_sentence_typing(text)
    text = normalize_acronyms(text)
    #text = word_segmentation(text) # When use PhoBERT
    text = remove_unnecessary_characters(text)
    # return text.lower()
    return text

In [17]:
path = r'C:\Users\Admin\Documents\Zalo Received Files\VLSP_Hotel'
os.chdir(path)
TRAIN_PATH = 'hotel_train.csv'
VAL_PATH = 'hotel_val.csv'
TEST_PATH = 'hotel_test.csv'

In [18]:
df_train = pd.read_csv(TRAIN_PATH).head(200)
df_val = pd.read_csv(VAL_PATH).head(50)
df_test = pd.read_csv(TEST_PATH).head(50)

In [19]:
df_val['Review'] = df_val['Review'].apply(lambda x: text_preprocess(str(x)))
df_test['Review'] = df_test['Review'].apply(lambda x: text_preprocess(str(x)))
df_train['Review'] = df_train['Review'].apply(lambda x: text_preprocess(str(x)))

In [20]:
def unpivot_and_filter(df):
    # Unpivot DataFrame
    df_unpivoted = pd.melt(df, id_vars=['Review'], var_name='Aspect', value_name='Sentiment')
    
    # Loại bỏ các dòng có sentiment "none"
    df_filtered = df_unpivoted #[df_unpivoted['Sentiment'] != 0]
    
    return df_filtered

# Áp dụng function cho các DataFrame
df_train = unpivot_and_filter(df_train).reset_index()
df_val = unpivot_and_filter(df_val).reset_index()
df_test = unpivot_and_filter(df_test).reset_index()

In [21]:
from torch.utils.data import DataLoader, Dataset
from transformers import RobertaTokenizer
import torch

tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

class CustomDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.max_len = max_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        review = str(self.data.loc[index, 'Review'])
        aspect = str(self.data.loc[index, 'Aspect'])
        sentiment = self.data.loc[index, 'Sentiment']
        
        # Encode sentiment
        if sentiment == 1: one_hot = [0, 1, 0, 0] # Positive
        elif sentiment == 2: one_hot = [0, 0, 1, 0] # Negatvie
        elif sentiment == 3: one_hot = [0, 0, 0, 1] # Neutral
        else: one_hot = [1, 0, 0, 0]  # Default to none if sentiment is missing or incorrect
            
        encoding = self.tokenizer.encode_plus(
            review + " " + aspect,  # Concatenate review and aspect
            max_length=self.max_len,
            add_special_tokens=True,
            return_token_type_ids=True,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'token_type_ids': encoding['token_type_ids'].flatten(),
            'labels': torch.tensor(one_hot, dtype=torch.float)
        }

# Tạo DataLoader
batch_size = 4
max_len = 256
train_dataset = CustomDataset(df_train, tokenizer, max_len=max_len)
val_dataset = CustomDataset(df_val, tokenizer, max_len=max_len)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

C:\Users\Admin\AppData\Roaming\Python\Python311\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [22]:
import torch.nn as nn
from transformers import RobertaModel

class CustomModel(nn.Module):
    def __init__(self, num_labels):
        super(CustomModel, self).__init__()
        self.roberta = RobertaModel.from_pretrained('roberta-base')
        self.drop = nn.Dropout(p=0.3)
        self.classifier = nn.Linear(self.roberta.config.hidden_size, num_labels)

    def forward(self, input_ids, attention_mask, token_type_ids):
        _, pooled_output = self.roberta(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            return_dict=False
        )
        output = self.drop(pooled_output)
        return self.classifier(output)

model = CustomModel(num_labels=4)  # Adjust num_labels to 4 for Positive, Negative, Neutral, None

import torch.optim as optim

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.AdamW(model.parameters(), lr=2e-5)

# Move model to the device
device = torch.device('cpu')
model.to(device)

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


CustomModel(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNo

In [23]:
# In ra một batch để kiểm tra
for batch in train_loader:
    print(batch)
    break

{'input_ids': tensor([[    0,   565, 10456,  ...,     1,     1,     1],
        [    0,   448,  3849,  ...,  3070,  6248,     2],
        [    0,   487,   298,  ...,     1,     1,     1],
        [    0, 17297,  3849,  ...,     1,     1,     1]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'labels': tensor([[1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.]])}


In [24]:
import torch.optim as optim

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.AdamW(model.parameters(), lr=2e-5)

def train(model, train_loader, criterion, optimizer, epochs):
    model.train()
    train_loss_history = []
    for epoch in range(epochs):
        running_loss = 0.0
        for i, batch in enumerate(train_loader):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            token_type_ids = batch['token_type_ids'].to(device)
            labels = batch['labels'].to(device)

            optimizer.zero_grad()
            outputs = model(input_ids, attention_mask, token_type_ids)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            if (i + 1) % 10 == 0:  # Print every 10 batches
                print(f"Epoch [{epoch+1}/{epochs}], Step [{i+1}/{len(train_loader)}], Loss: {loss.item():.4f}")

        epoch_loss = running_loss / len(train_loader)
        train_loss_history.append(epoch_loss)
        print(f"Epoch [{epoch+1}/{epochs}] finished with average loss: {epoch_loss:.4f}")

    return train_loss_history

def evaluate(model, val_loader, criterion):
    model.eval()
    val_loss_history = []
    val_loss = 0.0
    with torch.no_grad():
        for batch in val_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            token_type_ids = batch['token_type_ids'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids, attention_mask, token_type_ids)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
    epoch_val_loss = val_loss / len(val_loader)
    val_loss_history.append(epoch_val_loss)
    print(f"Validation Loss: {epoch_val_loss:.4f}")

    return val_loss_history



In [12]:
epochs = 3
train_loss_history = train(model, train_loader, criterion, optimizer, epochs)
val_loss_history = evaluate(model, val_loader, criterion)


KeyboardInterrupt: 

In [33]:

model_path = 'model_weights.pth'
torch.save(model.state_dict(), model_path)

In [ ]:
import torch
import torch.nn as nn
from transformers import BertTokenizer, BertForSequenceClassification
import re
model_name = "bert-base-uncased"  # Sử dụng mô hình BERT
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=34*4)
tokenizer = BertTokenizer.from_pretrained(model_name)

model.eval()

# Định nghĩa các danh mục từ dataframe đã unpivot
categories = ['FACILITIES#CLEANLINESS', 'FACILITIES#COMFORT', 'FACILITIES#DESIGN&FEATURES', 
              'FACILITIES#GENERAL', 'FACILITIES#MISCELLANEOUS', 'FACILITIES#PRICES', 
              'FACILITIES#QUALITY', 'FOOD&DRINKS#MISCELLANEOUS', 'FOOD&DRINKS#PRICES', 
              'FOOD&DRINKS#QUALITY', 'FOOD&DRINKS#STYLE&OPTIONS', 'HOTEL#CLEANLINESS', 
              'HOTEL#COMFORT', 'HOTEL#DESIGN&FEATURES', 'HOTEL#GENERAL', 'HOTEL#MISCELLANEOUS', 
              'HOTEL#PRICES', 'HOTEL#QUALITY', 'LOCATION#GENERAL', 'ROOM_AMENITIES#CLEANLINESS', 
              'ROOM_AMENITIES#COMFORT', 'ROOM_AMENITIES#DESIGN&FEATURES', 'ROOM_AMENITIES#GENERAL', 
              'ROOM_AMENITIES#MISCELLANEOUS', 'ROOM_AMENITIES#PRICES', 'ROOM_AMENITIES#QUALITY', 
              'ROOMS#CLEANLINESS', 'ROOMS#COMFORT', 'ROOMS#DESIGN&FEATURES', 'ROOMS#GENERAL', 
              'ROOMS#MISCELLANEOUS', 'ROOMS#PRICES', 'ROOMS#QUALITY', 'SERVICE#GENERAL']
# Hàm dự đoán
def predict(model, tokenizer, sentence):
    inputs = tokenizer.encode_plus(
        sentence,
        max_length=128,
        add_special_tokens=True,
        return_token_type_ids=True,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt'
    )

    input_ids = inputs['input_ids']
    attention_mask = inputs['attention_mask']
    token_type_ids = inputs['token_type_ids']

    with torch.no_grad():
        outputs = model(input_ids, attention_mask, token_type_ids)
        predictions = torch.sigmoid(outputs.logits).cpu().numpy()

    return predictions

# Hàm decode_predictions để giải mã các dự đoán và trả về danh sách các aspect và sentiment tương ứng
def decode_predictions(predicted_labels, threshold=0.5):
    sentiments = ['positive', 'negative', 'neutral', 'none']
    decoded_predictions = {}
    for i, pred in enumerate(predicted_labels[0]):
        if pred > threshold:
            sentiment = sentiments[i % 4]
            if sentiment in ['neutral', 'none']:
                continue  # Bỏ qua nếu sentiment là neutral hoặc none
            category_index = i // 4
            category = categories[category_index]
            if category not in decoded_predictions or pred > decoded_predictions[category][1]:
                decoded_predictions[category] = (sentiment, pred)
    
    return [f"{category}: {sentiment}" for category, (sentiment, _) in decoded_predictions.items()]

# Nhận văn bản đầu vào từ người dùng
input_text = input("Nhập văn bản: ")
example_sentence = text_preprocess(input_text)

# Dự đoán nhãn
predicted_labels = predict(model, tokenizer, example_sentence)

# Giải mã dự đoán
decoded_predictions = decode_predictions(predicted_labels)

# Hiển thị kết quả dự đoán
print(f"Example: {input_text}")
for prediction in decoded_predictions:
    print(f"=> {prediction}")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [34]:
import torch
import torch.nn as nn
from transformers import BertTokenizer, BertForSequenceClassification
import pandas as pd
import re
from sklearn.metrics import classification_report
# Định nghĩa mô hình và tokenizer
model_name = "bert-base-uncased"  # Sử dụng mô hình BERT
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=34*4)
tokenizer = BertTokenizer.from_pretrained(model_name)

model.eval()

# Định nghĩa các danh mục từ dataframe đã unpivot
categories = ['FACILITIES#CLEANLINESS', 'FACILITIES#COMFORT', 'FACILITIES#DESIGN&FEATURES', 
              'FACILITIES#GENERAL', 'FACILITIES#MISCELLANEOUS', 'FACILITIES#PRICES', 
              'FACILITIES#QUALITY', 'FOOD&DRINKS#MISCELLANEOUS', 'FOOD&DRINKS#PRICES', 
              'FOOD&DRINKS#QUALITY', 'FOOD&DRINKS#STYLE&OPTIONS', 'HOTEL#CLEANLINESS', 
              'HOTEL#COMFORT', 'HOTEL#DESIGN&FEATURES', 'HOTEL#GENERAL', 'HOTEL#MISCELLANEOUS', 
              'HOTEL#PRICES', 'HOTEL#QUALITY', 'LOCATION#GENERAL', 'ROOM_AMENITIES#CLEANLINESS', 
              'ROOM_AMENITIES#COMFORT', 'ROOM_AMENITIES#DESIGN&FEATURES', 'ROOM_AMENITIES#GENERAL', 
              'ROOM_AMENITIES#MISCELLANEOUS', 'ROOM_AMENITIES#PRICES', 'ROOM_AMENITIES#QUALITY', 
              'ROOMS#CLEANLINESS', 'ROOMS#COMFORT', 'ROOMS#DESIGN&FEATURES', 'ROOMS#GENERAL', 
              'ROOMS#MISCELLANEOUS', 'ROOMS#PRICES', 'ROOMS#QUALITY', 'SERVICE#GENERAL']
# Hàm chia văn bản dài thành các đoạn ngắn hơn
# Hàm chia văn bản dài thành các đoạn ngắn hơn
def split_text(text, max_length=128):
    words = text.split()
    chunks = [' '.join(words[i:i+max_length]) for i in range(0, len(words), max_length)]
    return chunks

# Hàm dự đoán cho từng đoạn văn bản
def predict(model, tokenizer, sentences, max_length=64):
    inputs = tokenizer.batch_encode_plus(
        sentences,
        max_length=max_length,
        add_special_tokens=True,
        return_token_type_ids=True,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt'
    )

    input_ids = inputs['input_ids']
    attention_mask = inputs['attention_mask']
    token_type_ids = inputs['token_type_ids']

    with torch.no_grad():
        outputs = model(input_ids, attention_mask, token_type_ids)
        predictions = torch.sigmoid(outputs.logits).cpu().numpy()

    return predictions

# Hàm decode_predictions để giải mã các dự đoán và trả về từ điển các aspect và sentiment tương ứng
def decode_predictions(predicted_labels, threshold=0.5):
    sentiments = ['positive', 'negative', 'neutral', 'none']
    decoded_predictions = {}
    for labels in predicted_labels:
        for i, pred in enumerate(labels):
            if pred > threshold:
                sentiment = sentiments[i % 4]
                if sentiment in ['neutral', 'none']:
                    continue  # Bỏ qua nếu sentiment là neutral hoặc none
                category_index = i // 4
                if category_index < len(categories):
                    category = categories[category_index]
                    if category not in decoded_predictions or pred > decoded_predictions[category][1]:
                        decoded_predictions[category] = (sentiment, pred)
    return decoded_predictions

# Giả sử cột chứa câu văn bản là 'Review' và nhãn thực tế là 'Aspect'

test_sentences = df_test['Review'].tolist()
y_test = df_test['Aspect'].tolist()  # Cột chứa nhãn thực tế

# Xử lý dữ liệu theo batch nhỏ
batch_size = 4  # Giảm kích thước batch
all_predictions = []
y_pred = []

for sentence in test_sentences:
    example_sentence = text_preprocess(sentence)
    chunks = split_text(example_sentence, max_length=32)  # Giảm max_length
    predictions = []
    for chunk in chunks:
        predicted_labels = predict(model, tokenizer, [chunk])
        predictions.append(predicted_labels[0])  # Lấy kết quả đầu tiên của batch
    merged_predictions = np.mean(predictions, axis=0)  # Tính trung bình các dự đoán
    decoded_predictions = decode_predictions([merged_predictions])
    all_predictions.append(decoded_predictions)
    y_pred.append(merged_predictions)  # Lưu lại kết quả dự đoán thô để đánh giá

# Thu thập nhãn thực tế và nhãn dự đoán cho sentiment
aspect_test = []
aspect_pred = []
sentiment_test = []
sentiment_pred = []

for row_test, row_pred in zip(y_test, y_pred):
    row_test_labels = row_test.split("; ")
    for index, col_pred in enumerate(row_pred):
        if index < len(categories):  # Kiểm tra giới hạn chỉ số
            if categories[index] in row_test_labels:
                aspect_test.append(categories[index])  # Nhãn thực tế
            if col_pred > 0.5:
                aspect_pred.append(categories[index])  # Nhãn dự đoán
    decoded_predictions = decode_predictions([row_pred])
    for category, (sentiment, _) in decoded_predictions.items():
        if category in row_test_labels:
            sentiment_test.append(sentiment)  # Sentiment thực tế (nếu có thông tin)
        sentiment_pred.append(sentiment)  # Sentiment dự đoán

# Đảm bảo rằng số lượng phần tử trong sentiment_test và sentiment_pred là nhất quán
min_length = min(len(sentiment_test), len(sentiment_pred))
sentiment_test = sentiment_test[:min_length]
sentiment_pred = sentiment_pred[:min_length]

# Đảm bảo rằng số lượng phần tử trong aspect_test và aspect_pred là nhất quán
min_length_aspect = min(len(aspect_test), len(aspect_pred))
aspect_test = aspect_test[:min_length_aspect]
aspect_pred = aspect_pred[:min_length_aspect]

# Đánh giá hiệu quả mô hình cho sentiment
if len(sentiment_test) > 0 and len(sentiment_pred) > 0:  # Kiểm tra danh sách không rỗng
    sentiment_report = classification_report(sentiment_test, sentiment_pred, digits=4, output_dict=True)
    print("Sentiment Classification Report")
    print(classification_report(sentiment_test, sentiment_pred, digits=4))  # Bảng đánh giá hiệu quả mô hình
else:
    print("No sentiment data to evaluate.")

# Đánh giá hiệu quả mô hình cho aspect
if len(aspect_test) > 0 and len(aspect_pred) > 0:  # Kiểm tra danh sách không rỗng
    aspect_report = classification_report(aspect_test, aspect_pred, digits=4, zero_division=1, output_dict=True)
    print("Aspect Classification Report")
    print(classification_report(aspect_test, aspect_pred, digits=4))  # Bảng đánh giá hiệu quả mô hình
else:
    print("No aspect data to evaluate.")

# Tạo DataFrame mới chứa review và dự đoán
df_predictions = pd.DataFrame({
    'Review': test_sentences,
    'Actual_Aspects': y_test,
    'Predicted_Sentiments': ["; ".join([f"{cat}: {sent}" for cat, (sent, _) in decode_predictions([pred]).items()]) for pred in y_pred]
})

# Điều chỉnh tùy chọn hiển thị của pandas để hiển thị đầy đủ cột 'Review'
pd.set_option('display.max_colwidth', None)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Sentiment Classification Report
              precision    recall  f1-score   support

    negative     0.6431    0.6431    0.6431       737
    positive     0.3490    0.3490    0.3490       404

    accuracy                         0.5390      1141
   macro avg     0.4961    0.4961    0.4961      1141
weighted avg     0.5390    0.5390    0.5390      1141

Aspect Classification Report
                                precision    recall  f1-score   support

        FACILITIES#CLEANLINESS     0.0366    0.0600    0.0455        50
            FACILITIES#COMFORT     0.0300    0.0600    0.0400        50
    FACILITIES#DESIGN&FEATURES     0.0000    0.0000    0.0000        50
            FACILITIES#GENERAL     0.0300    0.0600    0.0400        50
      FACILITIES#MISCELLANEOUS     0.0385    0.0400    0.0392        50
             FACILITIES#PRICES     0.0300    0.0600    0.0400        50
            FACILITIES#QUALITY     0.0300    0.0600    0.0400        50
     FOOD&DRINKS#MISCELLANEOUS     

C:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
